In [93]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

def your_code_here():
    return 1

adult_data = pd.read_csv("adult_with_pii.csv")
adult_data.head()

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [94]:
# Remove PII
adult_anon = adult_data.drop(columns=['Name', 'SSN'])
adult_anon.head()

,DOB,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,9/7/1967,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,6/7/1988,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,8/6/1991,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,4/6/2009,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,9/16/1951,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [95]:
# PII only
pii = adult_data[['Name', 'DOB', 'SSN', 'Zip']]
pii.head()

,Name,DOB,SSN,Zip
0,Karrie Trusslove,9/7/1967,732-14-6110,64152
1,Brandise Tripony,6/7/1988,150-19-2766,61523
2,Brenn McNeely,8/6/1991,725-59-9860,95668
3,Dorry Poter,4/6/2009,659-57-4974,25503
4,Dick Honnan,9/16/1951,220-93-3811,75387


## Question 1

(20 points)

Using the dataframes `pii` and `adult_anon`, perform a linking attack to recover the names of as many samples in `adult_anon` as possible.

How many names are you able to recover?

In [97]:
# In this cell, write code to perform the linking attack
x = adult_anon.groupby(["DOB", "Zip"]).size().reset_index(name="Count")
num_of_x_duplicates = x.loc[x["Count"] == 2]
y = pii.groupby(["DOB", "Zip"]).size().reset_index(name="Count")
num_of_y_duplicates = y.loc[y["Count"] == 2]

attack_df = pii.merge(right=adult_anon, how='inner', on=['DOB', 'Zip'])
attack_df = attack_df.drop_duplicates(subset=['SSN'], keep=False)

In [98]:
# In this cell, write code to determine how many names could be recovered

# Find common rows between 'attack_df' and 'adult_data'
df = adult_data.merge(attack_df, how='inner', indicator=False)

# Compare size of both DFs
print(f'Recovered {int(df.shape[0]/adult_data.shape[0]*100)}% of names ({df.shape[0]} of {adult_data.shape[0]})')

Recovered 99% of names (32557 of 32561)


## Question 2

(30 points)

Implement a function `is_k_anonymous` to check (for a given `k`) whether a given dataframe satisfies k-Anonymity.

In [99]:
# In this cell, write code to implement 'is_k_anonymous'

def is_k_anonymous(k, iqs, df):
    for identifier in iqs:
        if identifier in df.columns:
            value_counts = df[identifier].value_counts()
            if (value_counts > k).all():
                return True
                
    return False

## Question 3

(5 points)

In one or two sentences, informally describe how well you expect your implementation of 'is_k_anonymous' to scale with the size of the input data.

The implementation will scale quadratically with the input data. The first factor is the amount of quasi-identifiers we must inspect and the second
is the amount of data present for each quasi-identifier: O(n^2).

### Question 4 

(15 points)

Write code to answer the query: "how many participants have never been married?"

*Hint*: filter the `adult_data` dataframe to contain only participants who were never married, then return the 0th element of the `shape` of the filtered dataframe.

*Hint*: if you have not used Pandas before, [this](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html) comparison with SQL might be useful.

In [100]:
def never_married():
    
    marital_status = adult_data[adult_data['Martial Status'] == 'Never-married']
    return marital_status.shape[0]
    
query1 = never_married()
query1

10683

### Question 5 

(5 points)

In 2-5 sentences, answer the following:
- What is the sensitivity of `query1`, and why?

The sensitivity of `query1` is, on its own, seemingly unsensitive. However, when you compare it to a large database, you can extract a large nunber of insights. Marital status indicates quite a lot of information about a person and their lifestyle. For example, you can determine their likely age range by looking at general marriage statistics. Moreover, you may be able to target the spouse of the person you know if married if you have a different dataset that does not indicate their marital status.

### Question 6 

(15 points)

Define a function `laplace_mech` that implements the Laplace Mechanism for a query result consisting of a single real number. Your implementation should work for queries of any sensitivity (by passing sensitivity as an argument) and any value of `epsilon`.

*Hint*: use `np.random.laplace`.

In [101]:
def laplace_mech(query, sensitivity, epsilon):
    return query + np.random.laplace(scale= sensitivity / epsilon)

In [102]:
laplace_mech(48273, 1, 1)

48273.15199098369

### Question 7 

(10 points)

Use your implementation of `laplace_mech` to produce a differentially private answer to your query from the last question, with `epsilon = 0.1`.

In [103]:
laplace_mech(query1, 1, 0.1)

10685.64342653519